In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/ResearchProject/HAR/ppg+dalia/data/PPG_FieldStudy/S1/S1_E4/ACC.csv', header=0)

In [6]:
df.columns = ['acc_x', 'acc_y', 'acc_z']

In [7]:
df

,acc_x,acc_y,acc_z
0,32.0,32.0,32.0
1,-27.0,1.0,58.0
2,-27.0,2.0,59.0
3,-27.0,1.0,59.0
4,-27.0,1.0,58.0
...,...,...,...
298946,-24.0,0.0,60.0
298947,-24.0,-1.0,60.0
298948,-24.0,0.0,60.0
298949,-24.0,0.0,60.0


In [11]:
df["datetime"] = pd.to_datetime(df.index / 64, unit="s", origin="2024-01-01")


In [12]:
df

,acc_x,acc_y,acc_z,datetime
0,32.0,32.0,32.0,2024-01-01 00:00:00.000000000
1,-27.0,1.0,58.0,2024-01-01 00:00:00.015624960
2,-27.0,2.0,59.0,2024-01-01 00:00:00.031249920
3,-27.0,1.0,59.0,2024-01-01 00:00:00.046874880
4,-27.0,1.0,58.0,2024-01-01 00:00:00.062500096
...,...,...,...,...
298946,-24.0,0.0,60.0,2024-01-01 01:17:51.031249920
298947,-24.0,-1.0,60.0,2024-01-01 01:17:51.046874880
298948,-24.0,0.0,60.0,2024-01-01 01:17:51.062500096
298949,-24.0,0.0,60.0,2024-01-01 01:17:51.078125056


In [24]:
act_df = pd.read_csv('/content/drive/MyDrive/ResearchProject/HAR/ppg+dalia/data/PPG_FieldStudy/S1/S1_activity.csv', header=0)

In [25]:
act_df

,# SUBJECT_ID,S1
0,# NO_ACTIVITY,0
1,# BASELINE,90
2,# NO_ACTIVITY,791
3,# STAIRS,837
4,# NO_ACTIVITY,1125
5,# SOCCER,1210
6,# NO_ACTIVITY,1556
7,# CYCLING,1918
8,# NO_ACTIVITY,2331
9,# DRIVING,2935


In [26]:
act_df.columns = ['act_name', 'S1']

In [28]:
act_df['timedelta'] = pd.to_timedelta(act_df['S1'], unit = 's')

In [29]:
starting_time = pd.Timestamp('2024-01-01 00:00:00')
act_df['datetime'] = starting_time + act_df['timedelta']

In [35]:
act_df['activity'] = act_df['act_name'].apply(lambda x: 0 if x in ['# NO_ACTIVITY', '# BASELINE', '# DRIVING', '# LUNCH', "# CLEAN_BASELINE"] else 1)

In [36]:
act_df

,act_name,S1,timedelta,datetime,activity
0,# NO_ACTIVITY,0,0 days 00:00:00,2024-01-01 00:00:00,0
1,# BASELINE,90,0 days 00:01:30,2024-01-01 00:01:30,0
2,# NO_ACTIVITY,791,0 days 00:13:11,2024-01-01 00:13:11,0
3,# STAIRS,837,0 days 00:13:57,2024-01-01 00:13:57,1
4,# NO_ACTIVITY,1125,0 days 00:18:45,2024-01-01 00:18:45,0
5,# SOCCER,1210,0 days 00:20:10,2024-01-01 00:20:10,1
6,# NO_ACTIVITY,1556,0 days 00:25:56,2024-01-01 00:25:56,0
7,# CYCLING,1918,0 days 00:31:58,2024-01-01 00:31:58,1
8,# NO_ACTIVITY,2331,0 days 00:38:51,2024-01-01 00:38:51,0
9,# DRIVING,2935,0 days 00:48:55,2024-01-01 00:48:55,0


In [39]:
for i, row in act_df.iterrows():
    start_time = row['datetime']
    end_time = act_df.iloc[i + 1]['datetime'] if i < len(act_df) - 1 else pd.Timestamp('2024-01-01 23:59:59')
    activity_value = row['activity']
    df.loc[(df['datetime'] >= start_time) & (df['datetime'] < end_time), 'activity'] = activity_value

In [41]:
df.to_csv('/content/drive/MyDrive/ResearchProject/HAR/ppg+dalia/data/PPG_FieldStudy/S1/S1_E4/ACC_with_activity.csv')